In [2]:
from bs4 import BeautifulSoup
import pickle
import time
import requests
import csv
import re
from dateutil import parser
from datetime import timedelta
from datetime import datetime
import random
import os
import pandas as pd
import numpy as np

In [3]:
# Scrape a List of movies released in 2015 using wikipedia (and store the links)
movies = {}
years = [2013, 2014, 2015, 2016]

for year in years:
    response = requests.get("https://en.wikipedia.org/wiki/%d_in_film"%year)
    soup = BeautifulSoup(response.text, "lxml")
    tables = soup.find_all("table", class_="wikitable")

    for table in tables[2:8]: # The tables I want are 4 - 7
        for movie in table.find_all("i"):
            movie = movie.find("a")
            if movie is not None:
                url = movie.get("href").split("/")[-1].encode("utf-8")
                text = movie.text.encode("utf-8")
                movies[text] = {"url":url}

with open("movies.pkl", "w") as f:
    pickle.dump(movies, f)


/Library/Python/2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/Library/Python/2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/Library/Python/2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connecti

In [5]:
len(movies)

1009

In [6]:
# Grab finnancial data from box office mojo

def add_year(s):
    """Append the year to a date"""
    try:
        return s + "/%s"%str(year)[2:]
    except:
        return None
dfs = []
for year in years:
    for page in range(1,8):
        df = pd.read_html("http://www.boxofficemojo.com/yearly/chart/?page=%d&view=releasedate&view2=domestic&yr=%s&p=.htm"%(page,year))
        df = df[3].iloc[5:, 1:8]
        
        df.columns = ["title", "studio", "gross", "total theaters", "opening gross", "opening theaters", "release"]
        df.release = df.release.apply(add_year)
        dfs.append(df)
        time.sleep(1)    
    
    print "Done:", year

Done: 2013
Done: 2014
Done: 2015


IndexError: list index out of range

In [7]:
df = pd.concat(dfs, ignore_index=True) 

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2443 entries, 0 to 2442
Data columns (total 7 columns):
title               2371 non-null object
studio              2365 non-null object
gross               2371 non-null object
total theaters      2325 non-null object
opening gross       2156 non-null object
opening theaters    2173 non-null object
release             2299 non-null object
dtypes: object(7)
memory usage: 152.7+ KB


In [9]:
# Convert money values and date and save data
def strip_dollar_sign(s):
    try:
        return int(s.replace("$", "").replace(",",""))
    except (AttributeError, ValueError):
        return None
    
def to_datetime(t):
    try:
        return datetime.strptime(t, "%m/%d/%y")
    except:
        return None


df['opening gross'] = df['opening gross'].apply(strip_dollar_sign)
df['gross'] = df.gross.apply(strip_dollar_sign)
df.release = df.release.apply(to_datetime)
df['opening theaters'] = pd.to_numeric(df['opening theaters'], errors="coerce")
df['total theaters'] = pd.to_numeric(df['total theaters'], errors="coerce")

df.to_csv("finnancial.csv", encoding='utf-8', )


In [10]:
df.sample(10)

,title,studio,gross,total theaters,opening gross,opening theaters,release
1553,Chappie,Sony,31569268,3201,13346782,3201,2015-03-06
1884,Beasts of No Nation,BST,90777,31,51003,31,2015-10-16
1609,Dragon Ball Z: Resurrection 'F',FUN,8008363,913,1817937,183,2015-08-04
2322,Vaxxed: From Cover-Up to Catastrophe,Libre,118982,7,28339,1,2016-04-01
1989,Rediscover Sautet,Rialto,25314,1,NaN,NaN,2015-06-12
2256,Where to Invade Next,Drft.,3820195,308,897034,308,2016-02-12
1366,Empty Hours,Strand,8244,3,1366,1,2014-07-11
2306,Born to be Blue,EOne,151926,23,14069,23,2016-03-04
1872,Every Secret Thing,SM,103536,20,97020,20,2015-05-15
1557,Sinister 2,Focus,27740955,2799,10542116,2766,2015-08-21


In [16]:
from collections import defaultdict
import json
import urllib

wiki_movies = pickle.load(open("movies.pkl"))
wikiviews = pickle.load(open("wikiviews-Copy1.pkl"))

# Match box office names to wiki article names and use the stored links to find wiki traffic data
for movie_title, series in df.set_index('title').loc[[title for title in df.title if title in wiki_movies]].iterrows():
    article_name = wiki_movies[movie_title]['url']
    
    if movie_title not in wikiviews:
        release_date = series.release.date()
        temp_dict = defaultdict(dict)
        for date in [release_date, release_date - timedelta(weeks=4)]:
            date = str(date.year)+"%02d" % date.month
            response = requests.get('http://stats.grok.se/json/en/%s/%s'%(date, urllib.quote(article_name)))
            if response.ok:
                response = response.json()
                temp_dict['daily_views'].update(response['daily_views'])
            else:
                print "Warning! Failed:", movie_title, date
                continue
                
        temp_dict['title'] = response['title']
        wikiviews[movie_title] = temp_dict

        with open("log.txt", "a") as log:
            json.dump(response, log)
        
        print movie_title
        
        time.sleep(.5)



The Hunger Games: Catching Fire
Iron Man 3
Frozen
Despicable Me 2
Man of Steel
Gravity
Monsters University
The Hobbit: The Desolation of Smaug
Fast & Furious 6
Star Trek Into Darkness
Thor: The Dark World
World War Z
The Croods
The Heat
We're the Millers
American Hustle
The Conjuring
Identity Thief
Grown Ups 2
The Wolverine
Anchorman 2: The Legend Continues
Lone Survivor
G.I. Joe: Retaliation
Cloudy with a Chance of Meatballs 2
Now You See Me
The Wolf of Wall Street
The Hangover Part III
Epic
Captain Phillips
Jackass Presents: Bad Grandpa
Pacific Rim
Olympus Has Fallen
42
Elysium
Planes
The Lone Ranger
Oblivion
Saving Mr. Banks
Turbo
2 Guns
White House Down
Mama
Safe Haven
The Smurfs 2
The Best Man Holiday
Percy Jackson: Sea of Monsters
A Good Day to Die Hard
Warm Bodies
Jack the Giant Slayer
The Purge
Last Vegas
Ender's Game
Prisoners
After Earth
The Secret Life of Walter Mitty
12 Years a Slave
Free Birds
Hansel and Gretel: Witch Hunters
The Call
Gangster Squad
Jurassic Park 3D
The In

In [18]:
pickle.dump(wikiviews, open("wikiviews.pkl", "w"))

In [17]:
len(wikiviews)

595